In [1]:
%load_ext google.cloud.bigquery

In [2]:
import numpy as np 
import pandas as pd
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/shuang/Downloads/bqair-2b59936a9500.json"

# Big query helpers
from google.cloud import bigquery
from bq_helper import BigQueryHelper

# Import plotting libaries
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.io as pio
import geopandas
import shapely

# the Plotly in offline mode we can see map in the notebook.
from plotly.offline import download_plotlyjs,init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)

In [3]:
import plotly
plotly.tools.set_credentials_file(username='cleopathy', api_key='Vt9pQQVXJcV8bqHarLVt')

In [4]:
# getting pm25 data:
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        state_name,
        county_name,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.pm25_frm_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) in (2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017)
    
        """

In [5]:
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_pm25 = bq_assistant.query_to_pandas(QUERY) # getting data through big query to pandas
df_pm25['date_local'] = pd.to_datetime(df_pm25['date_local'])
df_pm25['year'] = df_pm25['date_local'].dt.year
df_pm25['month'] = df_pm25['date_local'].dt.month
import calendar
df_pm25['month_name'] = df_pm25['month'].apply(lambda x: calendar.month_abbr[x])
df_pm25['aqi'].fillna(df_pm25['aqi'].mean(), inplace = True)
df_pm25.head()

,state_code,county_code,date_local,arithmetic_mean,state_name,county_name,aqi,year,month,month_name
0,78,010,2004-06-02,5.4,Virgin Islands,St Croix,23.0,2004,6,Jun
1,78,010,2004-12-05,1.7,Virgin Islands,St Croix,7.0,2004,12,Dec
2,78,030,2006-10-14,7.2,Virgin Islands,St Thomas,30.0,2006,10,Oct
3,78,030,2004-06-02,7.0,Virgin Islands,St Thomas,29.0,2004,6,Jun
4,78,010,2004-01-16,2.2,Virgin Islands,St Croix,9.0,2004,1,Jan


In [8]:
df_pm25.info() 
df_pm25.drop_duplicates()
df_pm25.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1844133 entries, 0 to 44132
Data columns (total 10 columns):
state_code         object
county_code        object
date_local         datetime64[ns]
arithmetic_mean    float64
state_name         object
county_name        object
aqi                float64
year               int64
month              int64
month_name         object
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 154.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1844133 entries, 0 to 44132
Data columns (total 10 columns):
state_code         object
county_code        object
date_local         datetime64[ns]
arithmetic_mean    float64
state_name         object
county_name        object
aqi                float64
year               int64
month              int64
month_name         object
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 154.8+ MB


In [9]:
tg_pm25 = df_pm25
tg_pm25_avg = tg_pm25.groupby('state_name')['aqi'].mean()
tg_pm25_avg = tg_pm25_avg.reset_index()
tg_pm25_avg.aqi = tg_pm25_avg.aqi.round()
tg_pm25_avg.head()

,state_name,aqi
0,Alabama,45.0
1,Alaska,34.0
2,Arizona,32.0
3,Arkansas,43.0
4,California,41.0


In [10]:
import sklearn 

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [12]:
#now we would like to predict AQI of pm25 using sklearn,but now we have to use pandas get dummies and one-hot encoder.

In [13]:
tg_pm25_avg['state_name'].value_counts()

Louisiana               1
South Carolina          1
Texas                   1
Wyoming                 1
New Jersey              1
Canada                  1
Michigan                1
Florida                 1
Alabama                 1
Indiana                 1
Puerto Rico             1
Montana                 1
Rhode Island            1
Ohio                    1
Missouri                1
Massachusetts           1
Maine                   1
Connecticut             1
Minnesota               1
Mississippi             1
Delaware                1
Illinois                1
Pennsylvania            1
New York                1
California              1
West Virginia           1
Virgin Islands          1
Virginia                1
Utah                    1
Washington              1
North Carolina          1
Tennessee               1
Idaho                   1
Iowa                    1
Alaska                  1
Colorado                1
Kentucky                1
Georgia                 1
Nevada      

In [15]:
le = preprocessing.LabelEncoder()

In [16]:
le.fit(tg_pm25_avg['state_name'])

LabelEncoder()

In [17]:
list(le.classes_)

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Canada',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District Of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virgin Islands',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [20]:
state = le.transform(tg_pm25_avg['state_name'])

In [21]:
state

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53])

In [ ]:
#we write a function on categorical column encodering:
def 